### 依赖库
- numpy
- pytorch
- opencv-python
- PIL
- tqdm
- pandas
安装方式：略，百度，可用pip、conda，由于用到pytorch，需要安装cuda、cudnn，注意版本兼容。
本机 win10 + 1050.采用 cuda8.0 + cudnn7.0 并行计算


In [1]:
import sys
import os
import numpy as np
import torch
import torch.utils.data as data
import cv2
from PIL import Image
from tqdm import tqdm

### 首先看数据处理
数据：https://pan.baidu.com/s/1iA-7JCBff1ZE8PGUpVks6w 密码: sewi
下载下来是两个压缩包：test1.zip  train.zip
解压这两个压缩包，得到两个文件夹（可能存在部分图片解压缩失败，忽略）
- test1：测试图片所在目录
- train：训练数据集，子目录就是图片的标注—即图片的文字，共100个类x400个单类样本，共计400000个图像组成训练集
- label-test1-fake.csv：这是一个结果样例。作用是给出结果的标准格式。每一行是一个测试样本对应的一个结果，一行分为两列，分别是样本文件名，样本文字结果的top5.逗号隔开两列


In [2]:
# 训练集与测试集所在目录
sep = os.sep
trainpath = 'train'
testpath = 'test1'

# 训练集是分子目录的，且子目录是类标号，这里得到所有子目录（类别）及其数目
categories = os.listdir(trainpath)
category_number = len(categories)

# 后续用到卷积网络，这里设置图片的大小，之后需要统一将图片都reshape到这个尺寸
img_size = (128, 128)

**整个数据加载、处理（reshape、merge）都封装到一个函数：trans_data().该函数将训练集数据处理后保存到.npy文件中,
在transData内部，遍历整个子目录集合，分别读取子目录中的文件,函数为：load_by_category(category_index)
**

In [3]:
def load_by_category(category_index):
    # 目录拼接
    path = trainpath + sep + categories[category_index]
    # 获取所有该目录（类别）的样本（图片文件）
    files = os.listdir(path)
    # 结果集
    samples = []
    # 遍历所有该类样本
    for file in files:
        # 目录拼接
        file = path + sep + file
        # 读取图片到np数组
        img = np.asarray(Image.open(file))
        # 将图片reshape到128*128
        img = cv2.resize(img, img_size)
        # 加入结果集
        samples.append(img)
    samples = np.array(samples)
    # 类标号：每一幅图片的结果都是一个长度为category_number的结果向量
    labels = np.zeros([len(samples), category_number], dtype=np.uint8)
    # 当前labels的类标号设置为1，通过位置确定类
    # 可以想象最后将所有的category的labels拼接起来应该是一个阶梯矩阵
    labels[:, category_index] = 1
    return samples, labels

In [4]:
def trans_data():
    # 创建一个空的n*128*128的数据集合,第一维表示样本索引，第二三维128*128表示数据
    # 这里*img_size是列表的元素解包，这样[-1,*img_size]等于[-1,128,128]，而不是[-1,(128,128)]
    all_samples = np.zeros([0, *img_size], dtype=np.uint8)
    # 创建一个空的n*100的结果（类标号）集合，第一维表示样本index，100表示该样本属于这100个类别的概率（训练集上是1和0）
    labels = np.reshape(np.array([], dtype=np.uint8), [0, category_number])
    # 遍历所有训练集子目录（类别），这里tqdm是一个可视化进度条的库
    for index in tqdm(range(category_number)):
        # 获取某一个类别的数据和标注
        data, label = load_by_category(index)
        # 将结果合并到最终结果集中
        all_samples = np.append(all_samples, data, axis=0)
        labels = np.append(labels, label, axis=0)
    # 保存数据
    np.save('data.npy', all_samples)
    np.save('label.npy', labels)

**对数据处理过程最好的理解，是看数据的维度（shape）**

这里补充tqdm库的简要说明
> Tqdm 是一个快速，可扩展的Python进度条，可以在 Python 长循环中添加一个进度提示信息，用户只需要封装任意的迭代器 tqdm(iterator)。


```python
from tqdm import tqdm  
# 基本用法
for i in tqdm(range(10000)):  
    sleep(0.01)  

# trange用法,trange（i）是tqdm（i）的一个特殊实例
for i in trange(100):  
        sleep(0.1)  

# 只要传入可迭代iterator的对象都可以
pbar = tqdm(["a", "b", "c", "d"])  
for char in pbar:  
    pbar.set_description("Processing %s" % char) 

# 也可手动更新
with tqdm(total=100) as pbar:  
    for i in range(10):  
        pbar.update(10) 

# 手动更新
pbar = tqdm(total=100)  
for i in range(10):  
    pbar.update(10)  
pbar.close()          
```


**至此，数据处理部分就算算完成了，之后做两个工作**
- 测试上述过程
- 为外部提供一个数据集相关的接口。即让其他py模块只处理它的逻辑，需要用到数据，就从本py脚本中调用接口得到数据


In [5]:
# 对外提供一个加载测试集的函数，在应用模型测试的时候调用该方法
def load_test_data():
    files = os.listdir(testpath)
    test_data_set = []
    for file in tqdm(files):
        file = testpath + sep + file
        img = np.asarray(Image.open(file))
        img = cv2.resize(img, img_size)
        test_data_set.append(img)
    test_data_set = np.array(test_data_set)
    return test_data_set

In [6]:
# 读取之前保存的文件，检查shape，看看是不是预期的，我这里样本数是39999，少了一个，有张图解压缩的时候出错了
if __name__ == '__main__':
    # trans_data()
    train_data = np.load('data.npy')
    train_labels = np.load('label.npy')
    print(train_data.shape, train_labels.shape)
    test_data = load_test_data()
    print(test_data.shape)

(39999, 128, 128) (39999, 100)


100%|██████████████████████████████████████████████████████| 10000/10000 [00:07<00:00, 1358.95it/s]


(10000, 128, 128)


### 向外提供的数据接口——TrainSet类
- TrainSet类的父类是Torch的一种抽象类，表示一个数据集。torch.utils.data.Dataset。继承该类需要实现其两个抽象方法（An abstract class representing a Dataset.All other datasets should subclass it. All subclasses should override``__len__``, that provides the size of the dataset, and ``__getitem__``, supporting integer indexing in range from 0 to len(self) exclusive.）
- TrainSet的构造方法中，读取“data.npy”“label.npy”分别获取全部的data和label。但每次使用的并不是全部数据训练，而是打乱之后的